In [ ]:
# Load the mnist dataset
import tensorflow as tf
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()

# Normalize the pixel values
train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255
test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

In [11]:
# Create a dataset iterator to pass the dataset to the model in batches
BATCH_SIZE = 128
dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).batch(BATCH_SIZE)
iter = dataset.make_one_shot_iterator()
el = iter.get_next()

In [ ]:
# We need to define the weights and parameters for each of the NN layers, so make our lives easier by defining functions for doing this
def weight_variable(shape):
    # truncated_normal so that weights are not too far away from 0.0.
    initial = tf.truncated_normal( shape=shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    # small positive bias value so that we dont end with a lot of dead neurons using ReLU
    return tf.Variable(tf.constant(0.1, shape=shape))

# We are using Vanilla Convnets with stride 1, so define functions to do this automatically
def conv2d(input, filter):
    return tf.nn.conv2d(input, filter, strides=[1,1,1,1], padding="SAME")

def max_pool(value):
return tf.nn.max_pool(value, ksize=[1,2,2,1], strides=[1,2,2,1], padding="SAME")

In [ ]:
# First Layer: ConvNet with 32 filters of size 5*5 and ReLU activation followed by pooling
Wconv1 = weight_variable([5, 5, 1, 32]) # [filterDim1, filterDim2, filterDepth, NoOfFilters]
biasConv1 = bias_variable((32,))